In [1]:
import urllib3
from bs4 import BeautifulSoup
import requests
import certifi
import time
import datetime

In [2]:
#!/usr/bin/python
import MySQLdb

db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     #passwd="xpx",  # your password
                     db="xudatabase")        # name of the data base

cur = db.cursor()

In [3]:
dailyMax = 1000
batchNum = 20
sleeptimeforPubmed =5
backstep =20

cur.execute('select pmid from last_pmid')
endingPMID = max(cur.fetchall())
startingPMID=endingPMID[0] -backstep #



In [4]:
cur.execute('truncate articles')
# erase data or re-create articles table

0L

In [5]:
%%time
        for i in range (startingPMID, startingPMID+dailyMax, batchNum):
            PMIDs = ""

            for j in range (batchNum):
                PMIDs = PMIDs + str(i + j ) +","

            url = "https://www.ncbi.nlm.nih.gov/pubmed/"+ PMIDs +"?report=medline&format=text" 

            #time.sleep(sleeptimeforPubmed)
            http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
            r = http.request('GET', url)
            soup = BeautifulSoup(r.data, "lxml")

            allInfo = unicode(soup.pre.string)

            if allInfo.find('PMID- ')<0:
                print url
                print 'bad end'        
                break

            while allInfo.find('AB  -')>0:
                IdIndexStart = allInfo.index('PMID- ')+6
                IdIndexEnd = allInfo.index('\n', IdIndexStart)
                PMID = allInfo[IdIndexStart:IdIndexEnd]

                abIndexStart = allInfo.index('AB  -')+6
                abIndexEnd = allInfo.index('\n', abIndexStart)
                ab = allInfo[abIndexStart:abIndexEnd]

                query = "insert into articles values ("+ PMID +", '"+ ab + "')"
                endingPMID = int(PMID)
                if (endingPMID%10==0):
                    print endingPMID

                try:
                    cur.execute(query)
                    db.commit()

                except:
                    db.rollback()

                allInfo = allInfo[abIndexEnd:]

        now = datetime.datetime.now()       
        query = 'insert into last_pmid values ("%s", %d)'%(now, endingPMID)
        cur.execute(query)
        db.commit()
# write endingPMID to db
db.close()    
    

28288280
28288290
28288300
28288310
28288320
28288330
28288340
28288350
28288360
28288370
28288380
28288390
28288400
28288410
28288420
28288430
28288440
28288450
28288460
28288470
28288490
28288500
https://www.ncbi.nlm.nih.gov/pubmed/28288517,28288518,28288519,28288520,28288521,28288522,28288523,28288524,28288525,28288526,28288527,28288528,28288529,28288530,28288531,28288532,28288533,28288534,28288535,28288536,?report=medline&format=text
bad end
CPU times: user 324 ms, sys: 34.9 ms, total: 359 ms
Wall time: 10.7 s
